In [27]:
import sqlite3

# Connect to the SQLite database containing datasets
conn = sqlite3.connect('my_database.db')

# Create a cursor object
cursor = conn.cursor()

**SQL-Q1-What is the total number of drivers in the drivers dataset?**

In [28]:
# Execute an SQL query to count the number of drivers in the drivers table
cursor.execute('SELECT COUNT(*) FROM drivers')

# Fetch the result of the query
result = cursor.fetchone()

# Print the total number of drivers
print("Total number of drivers:", result[0])

Total number of drivers: 857


**SQL-Q2-Which constructor has won the most races?**

In [30]:

# Execute the SQL query to count the number of races for each constructor
cursor.execute('''SELECT constructors.name, COUNT(*) AS wins
                  FROM results
                  JOIN constructors ON results.constructorId = constructors.constructorId
                  WHERE results.position = 1
                  GROUP BY constructors.name
                  ORDER BY wins DESC''')

# Fetch the results of the query
results = cursor.fetchall()

# Print the results
for row in results:
    print(row[0], row[1])



Ferrari 243
McLaren 179
Mercedes 125
Williams 114
Red Bull 92
Team Lotus 45
Renault 35
Benetton 27
Tyrrell 23
Brabham 23
Lotus-Climax 22
BRM 17
Cooper-Climax 12
Lotus-Ford 11
Alfa Romeo 11
Vanwall 10
Matra-Ford 9
Maserati 9
Ligier 9
Brawn 8
Brabham-Repco 8
Kurtis Kraft 5
McLaren-Ford 4
Jordan 4
Wolf 3
Watson 3
March 3
Honda 3
Lotus F1 2
Epperly 2
Cooper-Maserati 2
Cooper 2
Brabham-Ford 2
Brabham-Climax 2
Toro Rosso 1
Stewart 1
Shadow 1
Racing Point 1
Porsche 1
Penske 1
Lotus-BRM 1
Kuzma 1
Hesketh 1
Eagle-Weslake 1
BMW Sauber 1
Alpine F1 Team 1
AlphaTauri 1


**SQL-Q3-Which drivers have the most podiums**

In [31]:

cursor.execute('''SELECT drivers.surname || ' ' || drivers.forename AS driver_name, COUNT(*) AS podiums
FROM results
JOIN drivers ON results.driverId = drivers.driverId
WHERE results.position IN (1, 2, 3)
GROUP BY driver_name
ORDER BY podiums DESC
LIMIT 5;''')

results = cursor.fetchall()

for row in results:
    print(row[0], row[1])


Hamilton Lewis 191
Schumacher Michael 155
Vettel Sebastian 122
Prost Alain 106
Räikkönen Kimi 103


**SQL-Q4-Which constructorst have the maximum points in each year?**

In [42]:
cursor.execute('''SELECT r.year, c.name AS constructor_name, MAX(cs.points) AS max_points
FROM constructor_standings cs
JOIN races r ON cs.raceId = r.raceId
JOIN constructors c ON cs.constructorId = c.constructorId
GROUP BY r.year
ORDER BY r.year DESC;
''')

results = cursor.fetchall()

for row in results:
    print(row[0], row[1], row[2])

2023 Alfa Romeo 0
2022 Alpine F1 Team 99
2021 Alpine F1 Team 95
2020 Renault 99
2019 Ferrari 99
2018 Haas F1 Team 93
2017 Mercedes 99
2016 Williams 96
2015 Red Bull 96
2014 Red Bull 99
2013 McLaren 95
2012 Force India 99
2011 Mercedes 98
2010 Renault 96
2009 Red Bull 98.5
2008 Ferrari 96
2007 BMW Sauber 94
2006 McLaren 97
2005 Ferrari 98
2004 Renault 96
2003 Ferrari 95
2002 Williams 92
2001 McLaren 95
2000 McLaren 98
1999 Ferrari 97
1998 Ferrari 92
1997 Williams 98
1996 Benetton 9
1995 Williams 98
1994 Benetton 97
1993 Williams 95
1992 McLaren 99
1991 McLaren 99
1990 McLaren 96
1989 McLaren 98
1988 McLaren 93
1987 Williams 93
1986 McLaren 96
1985 McLaren 90
1984 McLaren 92.5
1983 Brabham 9
1982 Team Lotus 9
1981 Williams 95
1980 Williams 90
1979 Ferrari 95
1978 Team Lotus 9
1977 Ferrari 95
1976 Ferrari 9
1975 Shadow-Ford 9.5
1974 McLaren 9
1973 Team Lotus 92
1972 Tyrrell 9
1971 Ferrari 9
1970 Brabham 9
1969 Matra-Ford 9
1968 Lotus-Ford 9
1967 Cooper-Maserati 9
1966 BRM 9
1965 Lotus-Cli

**SQL-Q5-Which constructors have the most consecutive wins? ( That is, has at least 1 winning race at that year )**

In [54]:
cursor.execute('''SELECT c.name AS constructor_name, COUNT(DISTINCT r.year) AS consecutive_wins
FROM constructor_standings cs1
JOIN constructors c ON cs1.constructorId = c.constructorId
JOIN races r ON cs1.raceId = r.raceId
WHERE cs1.position = 1
AND NOT EXISTS (
    SELECT 1
    FROM constructor_standings cs2
    WHERE cs2.constructorId = cs1.constructorId
    AND cs2.position != 1
    AND cs2.raceId = cs1.raceId + 1
)
GROUP BY c.name
ORDER BY consecutive_wins DESC;

''')

results = cursor.fetchall()

for row in results:
    print(row[0], row[1])

Ferrari 25
McLaren 22
Williams 11
Mercedes 8
Red Bull 6
Team Lotus 5
Renault 5
Lotus-Climax 3
BRM 3
Tyrrell 2
Cooper-Climax 2
Brabham-Repco 2
Benetton 2
Vanwall 1
Matra-Ford 1
March 1
Lotus-Ford 1
Ligier 1
Cooper 1
Brawn 1
Brabham 1
Alfa Romeo 1


**SQL-Q6-What is the average lap time of all drivers during the 2018 Monaco Grand Prix?**

In [57]:
cursor.execute('''SELECT AVG(lap_time) AS average_lap_time
FROM (
    SELECT AVG(milliseconds) AS lap_time
    FROM lap_times lt
    JOIN races r ON lt.raceId = r.raceId
    WHERE r.year = 2018 AND r.name = 'Monaco Grand Prix'
    GROUP BY lt.driverId, lt.lap
) AS subquery;

''')

results = cursor.fetchall()

print(results)

[(79857.7495042961,)]
